In [1]:
import pandas as pd
import string
from nltk.corpus import stopwords

In [35]:
df_spamcollection = pd.read_csv('/Users/suchitra/Desktop/work/private/course/Data science/datasciencepractice/dataset/spam.csv', usecols=['v1','v2'])

In [36]:
df_spamcollection.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [37]:
df_spamcollection.describe()

,v1,v2
count,5572,5572
unique,2,5169
top,ham,"Sorry, I'll call later"
freq,4825,30


In [38]:
df_spamcollection.groupby('v1').describe()

v2                                                               
     count unique                                                top freq
v1                                                                       
ham   4825   4516                             Sorry, I'll call later   30
spam   747    653  Please call our customer service representativ...    4

In [39]:
df_spamcollection['length'] = df_spamcollection['v2'].apply(len)

In [40]:
df_spamcollection.head()

,v1,v2,length
0,ham,"Go until jurong point, crazy.. Available only ...",111
1,ham,Ok lar... Joking wif u oni...,29
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155
3,ham,U dun say so early hor... U c already then say...,49
4,ham,"Nah I don't think he goes to usf, he lives aro...",61


In [41]:
def message_text_process(mess):
    no_punctuation = [char for char in mess if char not in string.punctuation]
    no_punctuation = ''.join(no_punctuation)
    return [word for word in no_punctuation.split() if word.lower() not in stopwords.words('english')]

In [42]:
df_spamcollection['v2'].head(5).apply(message_text_process)

0    [Go, jurong, point, crazy, Available, bugis, n...
1                       [Ok, lar, Joking, wif, u, oni]
2    [Free, entry, 2, wkly, comp, win, FA, Cup, fin...
3        [U, dun, say, early, hor, U, c, already, say]
4    [Nah, dont, think, goes, usf, lives, around, t...
Name: v2, dtype: object

In [43]:
from sklearn.feature_extraction.text import CountVectorizer


In [46]:
bag_of_words_transformer = CountVectorizer(analyzer=message_text_process).fit(df_spamcollection['v2'])

/Users/suchitra/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:4: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  after removing the cwd from sys.path.


In [47]:
print len(bag_of_words_transformer.vocabulary_)

11304


In [48]:
message_bagofwords = bag_of_words_transformer.transform(df_spamcollection['v2'])

/Users/suchitra/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:4: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  after removing the cwd from sys.path.


In [50]:
from sklearn.feature_extraction.text import TfidfTransformer
tfid_transformer = TfidfTransformer().fit(message_bagofwords)

In [53]:
message_tfidf = tfid_transformer.transform(message_bagofwords)
print message_tfidf.shape

 (5572, 11304)


In [60]:
from sklearn.naive_bayes import MultinomialNB
spam_detect_model =  MultinomialNB().fit(message_tfidf,df_spamcollection['v1'])

In [63]:
message = df_spamcollection['v2'][2]
bag_of_words_message = bag_of_words_transformer.transform(['v2'])
tfidf = tfid_transformer.transform(bag_of_words_message)

print 'predicted',spam_detect_model.predict(tfidf)[0]
print 'expected',df_spamcollection.v1[0]

predicted ham
expected ham
